In [ ]:
%%configure -f
{"defaultLakehouse": {"name": "lh_cfg"}}

In [ ]:
print("\n---------- Current notenook name: nb_system_cfg ----------\n")

In [ ]:
# kill me this and above cells

# Temp cell for emulating the parameters, passed by nb_master
is_debug = True

In [ ]:
# Dictionary to collect and return errors to the calling notebook
cfg_error = {}

In [ ]:
if is_debug: from pyspark.sql import SparkSession as ss

In [ ]:
# Print current session info
if is_debug: display(ss.getActiveSession())

In [ ]:
# Get ABFS Paths of the lakehouses in the current Workspace
try:
    ws_id = spark.conf.get("trident.workspace.id")
    lst = mssparkutils.lakehouse.list(workspaceId = ws_id)

    abfs_path_lh_cfg    = None
    abfs_path_lh_log    = None
    abfs_path_lh_bronze = None
    abfs_path_lh_silver = None

    for v in lst:
        if v.displayName == "lh_cfg":    abfs_path_lh_cfg    = v.properties["abfsPath"]
        if v.displayName == "lh_log":    abfs_path_lh_log    = v.properties["abfsPath"]
        if v.displayName == "lh_bronze": abfs_path_lh_bronze = v.properties["abfsPath"]
        if v.displayName == "lh_silver": abfs_path_lh_silver = v.properties["abfsPath"]

    if is_debug:
        print("----- ABFS Paths - Start -----")
        print(f"abfs_path_lh_cfg:    {abfs_path_lh_cfg}")
        print(f"abfs_path_llh_log:   {abfs_path_lh_log}")
        print(f"abfs_path_lh_bronze: {abfs_path_lh_bronze}")
        print(f"abfs_path_lh_silver: {abfs_path_lh_silver}")
        print("----- ABFS Paths - End -----")
except Exception as ex:
    cfg_error["Get ABFS Paths"] = str(ex)

In [ ]:
# Get values from table "lh_cfg.global_parameter"
try:
    sql_code = (f"""
SELECT DISTINCT `name`, `value`
FROM delta.`{abfs_path_lh_cfg}/Tables/global_parameter`
    """)

    sdf = spark.sql(sql_code)

    if is_debug:
        print(f"sql_code: {sql_code}")
        display(sdf)

    #abfs_path_lh_cfg                    = sdf.filter(sdf.name == "abfs_path_lh_cfg").select(sdf.value).collect()[0][0]
    #abfs_path_lh_log                    = sdf.filter(sdf.name == "abfs_path_lh_log").select(sdf.value).collect()[0][0]
    #abfs_path_lh_bronze                 = sdf.filter(sdf.name == "abfs_path_lh_bronze").select(sdf.value).collect()[0][0]
    #abfs_path_lh_silver                 = sdf.filter(sdf.name == "abfs_path_lh_silver").select(sdf.value).collect()[0][0]
    email_on_error = sdf.filter(sdf.name                      == "email_on_error").select(sdf.value).collect()[0][0]
    days_to_keep_log = sdf.filter(sdf.name                    == "days_to_keep_log").select(sdf.value).collect()[0][0]
    time_zone_nb = sdf.filter(sdf.name                        == "time_zone_nb").select(sdf.value).collect()[0][0]
    time_zone_pl = sdf.filter(sdf.name                        == "time_zone_pl").select(sdf.value).collect()[0][0]
    mssql_isolation_level = sdf.filter(sdf.name               == "mssql_isolation_level").select(sdf.value).collect()[0][0]
    projet_documentation_data_file_name = sdf.filter(sdf.name == "projet_documentation_data_file_name").select(sdf.value).collect()[0][0]

    if is_debug:
        print("----- lh_cfg.global parameter Start -----")
        #print(f"abfs_path_lh_cfg:                    {abfs_path_lh_cfg}")
        #print(f"abfs_path_lh_log:                    {abfs_path_lh_log}")
        #print(f"abfs_path_lh_bronze:                 {abfs_path_lh_bronze}")
        #print(f"abfs_path_lh_silver:                 {abfs_path_lh_silver}")
        print(f"email_on_error:                      {email_on_error}")
        print(f"days_to_keep_log:                    {days_to_keep_log}")
        print(f"time_zone_nb:                        {time_zone_nb}")
        print(f"time_zone_pl:                        {time_zone_pl}")
        print(f"mssql_isolation_level:               {mssql_isolation_level}")
        print(
            f"projet_documentation_data_file_name: {projet_documentation_data_file_name}"
        )
        print("----- lh_cfg.global parameter End -----")
except Exception as ex:
    cfg_error["Get global_parameter values"] = str(ex)

In [ ]:
# Print the config od the current session

import  os
from pyspark import SparkConf

spark_config = SparkConf().getAll()
for conf in spark_config:
    print(conf)